In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

import gc
import warnings
import datetime as dt
import math
import seaborn as sn

import tensorflow as tf # import tensorflow as a whole
from tensorflow import keras # only import Keras

# import the dense and input layer
from keras.layers import Dense, Input
# the sequential model object
from keras.models import Sequential

from numpy import linalg as LA
import math

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import tensorflow_addons as tfa

from keras.models import Sequential
from keras.layers import Input, CuDNNLSTM, Dense
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint



# load a saved model
from keras.models import load_model

In [2]:
#Import Data
data = pd.read_csv("data.csv")
samp_sol = pd.read_csv("sample_submission.csv")

In [3]:
data_set = pd.read_csv("data.csv")
display(data_set.head())

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [4]:
for k in data_set.columns:
    print("Column:", k)
    print(data_set[k].value_counts())
    print()

Column: id
0        1
21106    1
21104    1
21103    1
21102    1
        ..
10543    1
10542    1
10541    1
10540    1
31643    1
Name: id, Length: 31644, dtype: int64

Column: author
Thompson        17396
Janssen          2716
Weatherhead      2040
Beus             1604
Peskov           1084
Williams          891
Richenderfer      545
Mortimore         197
Kossolapov        101
Inasaka            46
Name: author, dtype: int64

Column: geometry
tube       21145
annulus     4381
plate        618
Name: geometry, dtype: int64

Column: pressure [MPa]
13.79    9226
6.89     4701
15.51    1117
10.34    1064
11.03     872
         ... 
3.77        1
12.21       1
7.06        1
7.70        1
12.14       1
Name: pressure [MPa], Length: 144, dtype: int64

Column: mass_flux [kg/m2-s]
4069.0    963
1519.0    634
1356.0    615
2034.0    533
1000.0    418
         ... 
835.0       1
967.0       1
6995.0      1
2050.0      1
1271.0      1
Name: mass_flux [kg/m2-s], Length: 733, dtype: int64

Column

In [5]:
max(np.array(data_set['x_e_out [-]']))

0.232

In [6]:
more_data = pd.read_csv("Data_CHF_Zhao_2020_ATE.csv")

In [7]:
for k in more_data.columns:
    print("Column:", k)
    print(more_data[k].value_counts())
    print()

Column: id
1       1
1240    1
1252    1
1251    1
1250    1
       ..
619     1
618     1
617     1
616     1
1865    1
Name: id, Length: 1865, dtype: int64

Column: author
Thompson        1202
Janssen          282
Weatherhead      162
Beus              77
Williams          51
Richenderfer      36
Mortimore         19
Peskov            17
Kossolapov        12
Inasaka            7
Name: author, dtype: int64

Column: geometry
tube       1439
annulus     378
plate        48
Name: geometry, dtype: int64

Column: pressure [MPa]
13.79    610
6.89     420
11.03     59
10.34     58
15.51     52
        ... 
10.57      1
10.93      1
12.34      1
12.38      1
10.24      1
Name: pressure [MPa], Length: 114, dtype: int64

Column: mass_flux [kg/m2-s]
4069    76
1519    55
2034    42
2292    28
1533    27
        ..
903      1
852      1
783      1
4164     1
7093     1
Name: mass_flux [kg/m2-s], Length: 578, dtype: int64

Column: x_e_out [-]
 0.0886    7
 0.0145    7
 0.0990    6
-0.0187    6
-0.

In [8]:
total_data = data_set.append(more_data).drop("id", axis=1)

/var/folders/tr/3zw9td7531z0_hhwc8105rv00000gn/T/ipykernel_74704/3739133671.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_data = data_set.append(more_data).drop("id", axis=1)


In [9]:
dummies_author = pd.get_dummies(total_data.author)
merged_with_authors = pd.concat([total_data, dummies_author], axis='columns')
merged_with_authors = merged_with_authors.drop(['author'], axis='columns')

dummies_geometry = pd.get_dummies(merged_with_authors.geometry)
merged_with_authors_and_geometry = pd.concat([merged_with_authors, dummies_geometry], axis='columns')
finalized_data = merged_with_authors_and_geometry.drop(['geometry'], axis='columns')

In [10]:
imputation = finalized_data.mean()

In [11]:
imputation = imputation.iloc[0]

In [12]:
final_test = finalized_data[pd.isnull(finalized_data["x_e_out [-]"])]
final_test = final_test.fillna(imputation)
final_test = final_test.drop("x_e_out [-]", axis=1)
finalized_data = finalized_data.dropna(subset=["x_e_out [-]"])
finalized_data = finalized_data.fillna(imputation)

In [13]:
submission_indices = list(final_test.index)

## Train / Test Split

In [14]:
X = finalized_data.drop("x_e_out [-]", axis=1)
y = finalized_data["x_e_out [-]"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [16]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [17]:
mu = np.mean(y_train)
sigma = np.std(y_train)

def transform(z):
    return (z - mu) / sigma

def inverse_transform(z):
    return (sigma * z) + mu

y_train = np.apply_along_axis(transform, 0, y_train)
y_test = np.apply_along_axis(transform, 0, y_test)

## Construct Model

In [18]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(19,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

2023-06-13 08:31:29.626450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
model.compile(loss=['mean_squared_error'] , optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1280      
                                                                 
 dense_1 (Dense)             (None, 8)                 520       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


In [20]:
#model = load_model('best_model_linear.h5')

In [21]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model_linear.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [22]:
model.fit(X_train, y_train, epochs=1000, batch_size=20, verbose=2, validation_split=0.1, callbacks=[es, mc])

Epoch 1/1000

Epoch 1: val_loss improved from inf to 2.95972, saving model to best_model_linear.h5
936/936 - 1s - loss: 660.0905 - accuracy: 0.0000e+00 - val_loss: 2.9597 - val_accuracy: 0.0000e+00 - 719ms/epoch - 769us/step
Epoch 2/1000

Epoch 2: val_loss improved from 2.95972 to 1.32010, saving model to best_model_linear.h5
936/936 - 0s - loss: 1.6137 - accuracy: 0.0000e+00 - val_loss: 1.3201 - val_accuracy: 0.0000e+00 - 445ms/epoch - 475us/step
Epoch 3/1000

Epoch 3: val_loss improved from 1.32010 to 1.08108, saving model to best_model_linear.h5
936/936 - 0s - loss: 1.2528 - accuracy: 0.0000e+00 - val_loss: 1.0811 - val_accuracy: 0.0000e+00 - 451ms/epoch - 482us/step
Epoch 4/1000

Epoch 4: val_loss improved from 1.08108 to 0.98068, saving model to best_model_linear.h5
936/936 - 0s - loss: 1.2447 - accuracy: 0.0000e+00 - val_loss: 0.9807 - val_accuracy: 0.0000e+00 - 446ms/epoch - 477us/step
Epoch 5/1000

Epoch 5: val_loss did not improve from 0.98068
936/936 - 0s - loss: 1.7668 - acc

Epoch 44/1000

Epoch 44: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9901 - accuracy: 0.0000e+00 - val_loss: 0.9916 - val_accuracy: 0.0000e+00 - 435ms/epoch - 465us/step
Epoch 45/1000

Epoch 45: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9925 - accuracy: 0.0000e+00 - val_loss: 0.9890 - val_accuracy: 0.0000e+00 - 436ms/epoch - 466us/step
Epoch 46/1000

Epoch 46: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9897 - accuracy: 0.0000e+00 - val_loss: 0.9908 - val_accuracy: 0.0000e+00 - 433ms/epoch - 462us/step
Epoch 47/1000

Epoch 47: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9879 - accuracy: 0.0000e+00 - val_loss: 0.9871 - val_accuracy: 0.0000e+00 - 433ms/epoch - 462us/step
Epoch 48/1000

Epoch 48: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9875 - accuracy: 0.0000e+00 - val_loss: 0.9821 - val_accuracy: 0.0000e+00 - 438ms/epoch - 468us/step
Epoch 49/1000

Epoch 49: val_loss did not improve from 0.80438
93

Epoch 88/1000

Epoch 88: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9799 - accuracy: 0.0000e+00 - val_loss: 0.9744 - val_accuracy: 0.0000e+00 - 430ms/epoch - 460us/step
Epoch 89/1000

Epoch 89: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9781 - accuracy: 0.0000e+00 - val_loss: 0.9739 - val_accuracy: 0.0000e+00 - 432ms/epoch - 461us/step
Epoch 90/1000

Epoch 90: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9822 - accuracy: 0.0000e+00 - val_loss: 0.9765 - val_accuracy: 0.0000e+00 - 434ms/epoch - 464us/step
Epoch 91/1000

Epoch 91: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9785 - accuracy: 0.0000e+00 - val_loss: 0.9858 - val_accuracy: 0.0000e+00 - 437ms/epoch - 467us/step
Epoch 92/1000

Epoch 92: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9801 - accuracy: 0.0000e+00 - val_loss: 0.9756 - val_accuracy: 0.0000e+00 - 433ms/epoch - 463us/step
Epoch 93/1000

Epoch 93: val_loss did not improve from 0.80438
93

Epoch 132/1000

Epoch 132: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9761 - accuracy: 0.0000e+00 - val_loss: 0.9829 - val_accuracy: 0.0000e+00 - 429ms/epoch - 459us/step
Epoch 133/1000

Epoch 133: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9781 - accuracy: 0.0000e+00 - val_loss: 0.9778 - val_accuracy: 0.0000e+00 - 429ms/epoch - 458us/step
Epoch 134/1000

Epoch 134: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9767 - accuracy: 0.0000e+00 - val_loss: 0.9802 - val_accuracy: 0.0000e+00 - 430ms/epoch - 459us/step
Epoch 135/1000

Epoch 135: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9766 - accuracy: 0.0000e+00 - val_loss: 0.9828 - val_accuracy: 0.0000e+00 - 430ms/epoch - 459us/step
Epoch 136/1000

Epoch 136: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9755 - accuracy: 0.0000e+00 - val_loss: 0.9877 - val_accuracy: 0.0000e+00 - 432ms/epoch - 462us/step
Epoch 137/1000

Epoch 137: val_loss did not improve fro

Epoch 176/1000

Epoch 176: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9720 - accuracy: 0.0000e+00 - val_loss: 0.9909 - val_accuracy: 0.0000e+00 - 434ms/epoch - 464us/step
Epoch 177/1000

Epoch 177: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9721 - accuracy: 0.0000e+00 - val_loss: 0.9812 - val_accuracy: 0.0000e+00 - 434ms/epoch - 464us/step
Epoch 178/1000

Epoch 178: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9736 - accuracy: 0.0000e+00 - val_loss: 0.9902 - val_accuracy: 0.0000e+00 - 435ms/epoch - 464us/step
Epoch 179/1000

Epoch 179: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9742 - accuracy: 0.0000e+00 - val_loss: 0.9763 - val_accuracy: 0.0000e+00 - 434ms/epoch - 463us/step
Epoch 180/1000

Epoch 180: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9737 - accuracy: 0.0000e+00 - val_loss: 0.9812 - val_accuracy: 0.0000e+00 - 431ms/epoch - 460us/step
Epoch 181/1000

Epoch 181: val_loss did not improve fro

Epoch 220/1000

Epoch 220: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9717 - accuracy: 0.0000e+00 - val_loss: 0.9836 - val_accuracy: 0.0000e+00 - 434ms/epoch - 463us/step
Epoch 221/1000

Epoch 221: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9728 - accuracy: 0.0000e+00 - val_loss: 0.9988 - val_accuracy: 0.0000e+00 - 435ms/epoch - 465us/step
Epoch 222/1000

Epoch 222: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9719 - accuracy: 0.0000e+00 - val_loss: 0.9890 - val_accuracy: 0.0000e+00 - 432ms/epoch - 461us/step
Epoch 223/1000

Epoch 223: val_loss did not improve from 0.80438
936/936 - 0s - loss: 0.9720 - accuracy: 0.0000e+00 - val_loss: 0.9856 - val_accuracy: 0.0000e+00 - 430ms/epoch - 460us/step
Epoch 223: early stopping


In [23]:
model = load_model('best_model_linear.h5')

In [24]:
y_hat = model.predict(X_test)

73/73 [==============================] - 0s 362us/step


In [25]:
math.sqrt(np.mean(np.square(y_test - y_hat)))# / math.sqrt(np.mean(np.square(y_test)))

1.1452654165277791

# Perform Prediction

In [26]:
X_submission = final_test

In [27]:
y_submission = model.predict(X_submission)
y_submission = np.apply_along_axis(inverse_transform, 0, y_submission)

326/326 [==============================] - 0s 284us/step


In [28]:
y_submission = list(y_submission.reshape(len(y_submission)))

In [29]:
df = pd.DataFrame(data={"id": submission_indices, "x_e_out [-]": y_submission})
df.to_csv("submission.csv", sep=',',index=False)